In [1]:
import requests
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote_plus
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import json
from collections import OrderedDict
import re
stopWord = ['ㅋ','ㅎ','^','!','ㅡ','-','_','ㅠ','ㅜ',';','ㅗ','ㄷ','\n']

ModuleNotFoundError: No module named 'selenium'

In [ ]:
def clean_text(text):
#    cleaned_text = re.sub('[a-zA-Z]' , '', text)
#    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
#                          '', cleaned_text)
#    return cleaned_text
    return text

In [2]:
import requests
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote_plus
import pandas as pd
import numpy as np


def crawling(keyword,n):
    error_count=0
    count=0
    base_url ='https://www.instagram.com/graphql/query/?query_hash=9b498c08113f1e09617a1703c22b2f32'
    
    resp = requests.get('https://www.instagram.com/explore/tags/'+keyword+'/?__a=1')
    edge = resp.json()["graphql"]["hashtag"]["edge_hashtag_to_media"]["edges"]
    has_next_page = resp.json()["graphql"]["hashtag"]["edge_hashtag_to_media"]['page_info']['has_next_page']
    end_cursor = resp.json()["graphql"]["hashtag"]["edge_hashtag_to_media"]['page_info']['end_cursor']
    n=1
    SAVE_DIR='./'+keyword+'/'
    if not os.path.exists(SAVE_DIR):
        os.mkdir(SAVE_DIR)
    
    for i in edge:
        count+=1
        try:
            text = i["node"]["edge_media_to_caption"]['edges'][0]['node']['text']
            code = i["node"]['shortcode']
            img = i['node']['display_url']
            dic = {'hashtag' : []}
            text_list = clean_text(i["node"]["edge_media_to_caption"]['edges'][0]['node']['text']).split('#')
            for text in text_list:
                if len(text)<10:
                    if text not in stopWord:
                        dic['hashtag'].append(clean_text(text).strip().replace('\n',''))
            df = pd.DataFrame(dic)
            df.to_csv(keyword+'/'+keyword+str(n)+'.csv')
            with urlopen(img) as f:
                with open(keyword+'/'+keyword+str(n)+'.jpg','wb') as c:
                    temp = f.read()
                    c.write(temp)
            n+=1
        except:
            pass

    params = {
            "tag_name":keyword,
            "first":4,
            "after": end_cursor
        }

    while has_next_page:
        params = {
            "tag_name":keyword,
            "first":4,
            "after": end_cursor
        }
        try:
            new_resp = requests.get(base_url,params=params)
            print(new_resp)
            edge = new_resp.json()["data"]["hashtag"]["edge_hashtag_to_media"]["edges"]
            has_next_page = new_resp.json()["data"]["hashtag"]["edge_hashtag_to_media"]['page_info']['has_next_page']
            end_cursor = new_resp.json()["data"]["hashtag"]["edge_hashtag_to_media"]['page_info']['end_cursor']
            for i in edge:
                dic = {'hashtag' : []}
                count+=1
                text_list = clean_text(i["node"]["edge_media_to_caption"]['edges'][0]['node']['text']).split('#')
                for text in text_list:
                    if len(text)<10:
                        if text not in stopWord:
                            dic['hashtag'].append(clean_text(text))
                            
                df = pd.DataFrame(dic)
                df.to_csv(keyword+'/'+keyword+str(n)+'.csv')
                code = i["node"]['shortcode']
                img = i['node']['display_url']
                with urlopen(img) as f:
                    with open(keyword+'/'+keyword+str(n)+'.jpg','wb') as c:
                        temp = f.read()
                        c.write(temp)
                n+=1
                if count>=n:
                    break
        except:
            error_count+=1
            if error_count>=10:
                return

In [3]:
crawling('초코칩',300)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

['83198561398', '53753박현규', '윤영선', '마준영123', '이영재elnelf', 'lfjnf']

asdf	asdf


In [61]:
#크롤링_해올_리스트 = ['윤아','태연','티파니','선미','한효주']
크롤링_해올_리스트 = ['스타벅스','해커스','엽떡','크리스마스']
for i in 크롤링_해올_리스트:
    crawling(i,300)